# Test on Velodyne data

In [ ]:
%matplotlib widget
import numpy as np
import os
import time
import plotly.graph_objects as go

from planeslam.general import plot_3D_setup, downsample
from planeslam.mesh import LidarMesh
from planeslam.scan import Scan
from planeslam.clustering import cluster_mesh_graph_search, plot_clusters
from planeslam.extraction import scan_from_clusters, planes_from_clusters
from planeslam.general import pc_plot_trace

%load_ext autoreload
%autoreload 2

Load data

In [ ]:
# Read in point cloud data
pcpath = os.path.join(os.getcwd(),'..', 'data', 'velodyne', '6_7_2022', 'durand_3rd_floor', 'run_1')

In [ ]:
# PCs = []
# for i in range(len(os.listdir(pcpath))):
#     filename = pcpath+'/pc_'+str(i)+'.npy'
#     PC = np.load(filename)
#     PCs.append(PC)

frame = 2000
filename = pcpath+'/pc_'+str(frame)+'.npy'
PC = np.load(filename)

In [ ]:
# Downsample
PC = downsample(PC, factor=5, axis=0)

In [ ]:
fig = go.Figure(data=pc_plot_trace(PC))
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

Cluster

In [ ]:
# Create the mesh
mesh = LidarMesh(PC)
mesh.prune(edge_len_lim=2)
start_time = time.time()
# Cluster the mesh with graph search
clusters, avg_normals = cluster_mesh_graph_search(mesh, normal_match_thresh=0.707)
print("elapsed time: ", time.time() - start_time)

In [ ]:
# Plot mesh
fig = go.Figure(data=mesh.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Plot clusters
plot_clusters(PC, mesh, clusters)

Extract planes

In [ ]:
# planes, vertices, faces = scan_from_clusters(mesh, clusters, avg_normals)
# scan = Scan(planes, vertices, faces)
planes = planes_from_clusters(mesh, clusters, avg_normals)
scan = Scan(planes)

In [ ]:
fig = go.Figure(data=scan.plot_trace())
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

Pclpy

In [ ]:
import pclpy
from pclpy import pcl
import math

a = np.random.rand(10,3)
pc = pcl.PointCloud.PointXYZ.from_array(PC)
pc.xyz

In [ ]:
rg = pcl.segmentation.RegionGrowing.PointXYZ_Normal()

rg.setInputCloud(pc)
normals_estimation = pcl.features.NormalEstimationOMP.PointXYZ_Normal()
normals_estimation.setInputCloud(pc)
normals = pcl.PointCloud.Normal()
normals_estimation.setRadiusSearch(0.5)
start_time = time.time()
normals_estimation.compute(normals)
print("elapsed time: ", time.time() - start_time)

In [ ]:
rg.setInputNormals(normals)
rg.setMaxClusterSize(1000000)
rg.setMinClusterSize(10)
rg.setNumberOfNeighbours(30)
rg.setSmoothnessThreshold(2 / 180 * math.pi)
rg.setCurvatureThreshold(1)
rg.setResidualThreshold(1)
start_time = time.time()
clusters = pcl.vectors.PointIndices()
rg.extract(clusters)
print("Elapsed time: ", time.time() - start_time)
print("Number of clusters: ", len(clusters))

In [ ]:
ax = plot_3D_setup(PC)

for i, c in enumerate(clusters):
    idx = c.indices
    ax.scatter3D(pc.xyz[idx,0], pc.xyz[idx,1], pc.xyz[idx,2], color='C'+str(i), marker='.')  
ax.set_box_aspect((np.ptp(pc.xyz[:,0]), np.ptp(pc.xyz[:,1]), np.ptp(pc.xyz[:,2])))
ax.set_xlabel("X")
ax.set_ylabel("Y")

Flight room

In [ ]:
# Read in point cloud data
pcpath = os.path.join(os.getcwd(),'..', 'data', 'velodyne', '8_5_2022', 'flightroom', 'test')
filename = pcpath + '/pc_11.npy'
PC = np.load(filename)

In [ ]:
fig = go.Figure(data=pc_plot_trace(PC))
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Downsample
PC = downsample(PC, factor=5, axis=0)

In [ ]:
# Create the mesh
mesh = LidarMesh(PC)
mesh.prune(edge_len_lim=2)
start_time = time.time()
# Cluster the mesh with graph search
clusters, avg_normals = cluster_mesh_graph_search(mesh, normal_match_thresh=0.707)
print("elapsed time: ", time.time() - start_time)

In [ ]:
# Plot mesh
fig = go.Figure(data=mesh.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Plot clusters
plot_clusters(PC, mesh, clusters)

In [ ]:
from planeslam.clustering import mesh_cluster_pts, sort_mesh_clusters

clusters, avg_normals = sort_mesh_clusters(clusters, avg_normals)
i = 0
n = avg_normals[i][:,None]
c = clusters[i]
cluster_pts = mesh_cluster_pts(mesh, c)  # Extract points from cluster

In [ ]:
# Plot the points 
fig = go.Figure(data=pc_plot_trace(cluster_pts))
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
planes = planes_from_clusters(mesh, clusters, avg_normals)
scan = Scan(planes)

In [ ]:
from planeslam.scan import pc_to_scan
scan = pc_to_scan(PC, ds_rate=5, edge_len_lim=2)

In [ ]:
fig = go.Figure(data=scan.plot_trace())
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
scan.remove_small_planes(area_thresh=0.1)

In [ ]:
fig = go.Figure(data=scan.plot_trace())
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
scan.reduce_inside(p2p_dist_thresh=0.1)

In [ ]:
fig = go.Figure(data=scan.plot_trace())
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
scan.fuse_edges()

In [ ]:
fig = go.Figure(data=scan.plot_trace())
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()